In [ ]:
import dropbox
import sys
import os

# Replace with your OAuth2 access token
ACCESS_TOKEN = ''
dbx = dropbox.Dropbox(ACCESS_TOKEN)

# Define the chunk size (4MB)
CHUNK_SIZE = 4 * 1024 * 1024

def upload_large_file(file_path, destination_path):
    try:

        file_size = os.path.getsize(file_path)
        
        print(f"Uploading {file_path} to Dropbox")
        print(f"File size: {file_size} bytes")
        print(f"Chunk size: {CHUNK_SIZE} bytes")

        with open(file_path, 'rb') as f:
            upload_session_start_result = dbx.files_upload_session_start(f.read(CHUNK_SIZE))
            cursor = dropbox.files.UploadSessionCursor(session_id=upload_session_start_result.session_id,
                                                       offset=f.tell())
            commit = dropbox.files.CommitInfo(path=destination_path)

            while f.tell() < file_size:
                if (file_size - f.tell()) <= CHUNK_SIZE:
                    print(f"Uploading last chunk of {file_path}")
                    dbx.files_upload_session_finish(f.read(CHUNK_SIZE),
                                                    cursor,
                                                    commit)
                else:
                    print(f"Uploading chunk of {file_path}")
                    dbx.files_upload_session_append(f.read(CHUNK_SIZE),
                                                    cursor.session_id,
                                                    cursor.offset)
                    cursor.offset = f.tell()
                
                # Print progress
                progress = (f.tell() / file_size) * 100
                print(f"Upload progress: {progress:.2f}%")

        print(f"File uploaded successfully to {destination_path}")
    except Exception as e:
        print(f"Error uploading file: {str(e)}")


    
file = 'test.txt'
local_path = f'./{file}'
dropbox_path = f'/ML datasets/{file}'


if not os.path.exists(local_path):
    print(f"Error: File {local_path} does not exist.")
    sys.exit(1)

upload_large_file(local_path, dropbox_path)


In [5]:
# download
import dropbox


# Replace with your OAuth2 access token
ACCESS_TOKEN = ''
dbx = dropbox.Dropbox(ACCESS_TOKEN)

file = 'test.txt'
local_path = f'./{file}'
dropbox_path = f'/{file}'
# dropbox_path = f'/ML datasets/Ford/2017-10-26/V2/Log1/{file}'


dbx.files_download_to_file(local_path, dropbox_path)


FileMetadata(client_modified=datetime.datetime(2024, 9, 28, 21, 35, 31), content_hash='5a2c4ec5dbb1370fefba414b81cc7b7d209e923a4c420f74fb2bf0cc3a305719', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:9Sm55dbVQTgAAAAAAAAMBw', is_downloadable=True, media_info=NOT_SET, name='test.txt', parent_shared_folder_id=NOT_SET, path_display='/test.txt', path_lower='/test.txt', preview_url=NOT_SET, property_groups=NOT_SET, rev='62334c0819aea73fb1b51', server_modified=datetime.datetime(2024, 9, 28, 21, 35, 35), sharing_info=NOT_SET, size=16, symlink_info=NOT_SET)

In [ ]:




dbx.users_get_current_account()
for entry in dbx.files_list_folder('/ML datasets').entries:
    print(entry.name)

file = 'test.txt'
local_path = './' + file
dropbox_path = '/ML datasets/'+file
print(dropbox_path)